### Attempting to remove stopwords before splitting the line lyrics by Token
#### Unsuccessful

In [1]:
# Import libraries and upload OHCO (by line) #
import pandas as pd
import numpy as np
import os
os.chdir('/Users/nickbruno/Documents/spring_2019/DS5559/project/code')

df = pd.read_csv('artist_song_verse_line_OHCO_df.csv') # includes corpus with no duplicate song names

In [2]:
df.head()

,artist_id,song_id,verse_num,line_num,Line
0,0,0,0,0,"Look at her face, it's a wonderful face"
1,0,0,0,1,And it means something special to me
2,0,0,0,2,Look at the way that she smiles when she sees me
3,0,0,0,3,How lucky can one fellow be?
4,0,0,1,0,"She's just my kind of girl, she makes me feel ..."


In [22]:
# Try the method on a subset first
sub = df.iloc[:5]

In [27]:
# Remove stopwords #
from nltk.corpus import stopwords
stop = stopwords.words('english')

# sub['mod_line'] = sub['Line'].str.split(' ').apply(lambda x: ' '.join(k for k in x if k not in stop))
sub['Line'] = sub['Line'].str.replace('\d', '')
    # works on a substring

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [24]:
sub.head()

,artist_id,song_id,verse_num,line_num,Line
0,0,0,0,0,"Look at her face, it's a wonderful face"
1,0,0,0,1,And it means something special to me
2,0,0,0,2,Look at the way that she smiles when she sees me
3,0,0,0,3,How lucky can one fellow be?
4,0,0,1,0,"She's just my kind of girl, she makes me feel ..."


In [29]:
# Apply this to the entire corpus 
df['mod_line'] = df['Line'].str.split(' ').apply(lambda x: ' '.join(k for k in x if k not in stop))
    # Doesn't work because it can not iterate through 'float' or 'int' values

TypeError: 'float' object is not iterable

### We cannot remove stopwords before creating the Token table because it cannot iterate through numbers

In [16]:
# Define a function to find numbers
def num_there(s):
    return any(i.isdigit() for i in s)

In [18]:
num_there(sub.lyrics) # indicates that there are no digits in the line lyrics

False

In [ ]:
# Takes too long to run #
TOKEN_PAT = r'(\W+)'
tokens = df.Line.str.split(TOKEN_PAT, expand=True)\
    .stack()\
    .to_frame()\
    .rename(columns={0:'token_str'})